In [174]:
import cv2
import copy
import time
import pathlib
import random
import warnings
import datetime
import scipy as sp
import pandas as pd
import numpy as np
from time import time
import lightgbm as lgb
from catboost import Pool
from functools import partial
import matplotlib.pyplot as plt
from catboost import CatBoost
from sklearn import preprocessing
from tqdm import tqdm_notebook as tqdm
from collections import Counter, defaultdict
from scipy.interpolate import UnivariateSpline
from hyperopt import hp, tpe, Trials, fmin, space_eval
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, average_precision_score
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=5)
warnings.filterwarnings("ignore")

In [33]:
def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

In [93]:
filepath = "../../../../pythonwork/20200529atma/input/"
train = pd.read_csv(filepath + "train.csv")
test = pd.read_csv(filepath + "test.csv")
fitting = pd.read_csv(filepath + "fitting.csv")
sample_submission = pd.read_csv(filepath + "atmacup5__sample_submission.csv")
spec_df = pd.read_csv(filepath + "spec.csv")

# FE

In [181]:
def transform(df):
    new_df = df.copy()
    # merge original csv and fitting data
    new_df = pd.merge(new_df, fitting, on="spectrum_id", how="left")
    
    # aggragate spec information
    spec_agg = spec_df.groupby("spectrum_filename")["intensity"].agg(["max", "min", "mean", "std"])
    spec_agg.columns = ["intensity_" + c for c in spec_agg.columns]
    #root_len = []
    #for i, file_df in spec_df.groupby("spectrum_filename"):
     #   x = file_df["wavelength"].values
      #  y = file_df["intensity"].values
       # spline = UnivariateSpline(x, y-np.max(y)/2, s=0)
        #roots = spline.roots()
        #root_max.append(y)
        #root_len.append(len(roots))
    #spec_agg["root_len"] = root_len
    
    # merge original csv and spec information
    new_df= pd.merge(new_df, spec_agg.reset_index(), on="spectrum_filename", how="left")
    
    # remove unnecessary columns
    if "target" in new_df.columns:
        new_df = new_df.drop(["spectrum_id",  "spectrum_filename"], axis=1)
    else:
        new_df = new_df.drop(["spectrum_id",  "spectrum_filename","chip_id"], axis=1)
    
    # create new variables

    return new_df
new_train = transform(train)
new_test = transform(test)
print(f'train shape: {new_train.shape}')
print(f'test shape: {new_test.shape}')

train shape: (7436, 20)
test shape: (6952, 18)


In [182]:
new_train.head()

,chip_id,exc_wl,layout_a,layout_x,layout_y,pos_x,target,params0,params1,params2,params3,params4,params5,params6,rms,beta,intensity_max,intensity_min,intensity_mean,intensity_std
0,79ad4647da6de6425abf,850,2,36,140,1313.0810,0,30.808589,5.811802e+02,1037.714752,1.531423,22469.651641,1032.317268,8.295610,10.028668,2.521298e-02,1751.0,-228.0,40.292752,172.206792
1,79ad4647da6de6425abf,780,3,0,168,159.4150,0,91.300897,1.740582e+04,1080.510452,4.766233,33257.123175,1077.468855,8.018225,7.948485,3.435612e-01,4219.0,-263.0,166.958984,463.428363
2,c695a1e61e002b34e556,780,1,34,29,-610.7688,0,106.642946,1.000000e-10,1119.464438,2.000000,42579.867913,1378.883338,11.687417,10.739859,2.348528e-15,2412.0,-235.0,151.577691,327.857694
3,c695a1e61e002b34e556,780,2,32,139,1214.6180,0,306.933674,1.099486e+04,1139.855067,5.198692,39349.741703,1145.212849,9.445029,10.379948,2.183921e-01,3209.0,-52.0,523.080947,436.481410
4,c695a1e61e002b34e556,780,0,45,85,-257.6161,0,46.133256,2.227622e+04,1120.918337,5.668012,31054.928673,1117.107782,7.658710,8.316550,4.176962e-01,3998.0,-245.0,138.187717,472.009931


# modelling

## lgb

In [190]:
categoricals = ['exc_wl'] #, 'layout_a',
lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.05, 'random_seed':44,'max_depth': 3}

def modelling(new_train, new_test, lgbm_params):
    X_train = new_train.drop(['target'],axis=1).copy()
    y_train = new_train.target.copy()
    
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["chip_id"]))
    X_train["chip_id"] = lbl.transform(list(X_train["chip_id"]))
    
    remove_features = ["layout_a"]
    for i in X_train.columns:
        if (X_train[i].std() == 0) and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_test = new_test.copy()
    X_test = X_test.drop(remove_features, axis=1)

    n_folds=6
    skf=GroupKFold(n_splits = n_folds)
    models = []

    valid = np.array([])
    valid_lgb = pd.DataFrame(np.zeros([X_train.shape[0]]))
    real = np.array([])
    features_list = [i for i in X_train.columns if i != "chip_id"]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    initial = lgbm_params["random_seed"]
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["chip_id"])):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        
        X_train2.drop(["chip_id"], axis=1, inplace=True)
        X_test2.drop(["chip_id"], axis=1, inplace=True)
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=10,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals)
        valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        valid = np.concatenate([valid, valid_predict])
        valid_lgb.iloc[test_index]  = clf.predict(X_test2, num_iteration = clf.best_iteration).reshape(X_test2.shape[0], 1)
        real = np.concatenate([real, y_test2])
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()

        models.append(clf)
        
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

    score = average_precision_score(real, valid)
    print("average precision score = {}".format(average_precision_score(real, valid)))
    print(confusion_matrix(real, np.round(valid)))
    pred_value = np.zeros(X_test.shape[0])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return score, pred_value, feature_importance_df, valid_lgb

metric, pred_value, feature_importance_df, _ = modelling(new_train, new_test, lgbm_params)

Fold 1
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[213]	training's pr_auc: 0.995292	valid_1's pr_auc: 0.817128
Fold 2
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[48]	training's pr_auc: 0.865579	valid_1's pr_auc: 0.828095
Fold 3
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[84]	training's pr_auc: 0.901349	valid_1's pr_auc: 0.86098
Fold 4
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[33]	training's pr_auc: 0.806856	valid_1's pr_auc: 0.677849
Fold 5
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[123]	training's pr_auc: 0.9519	valid_1's pr_auc: 0.622129
Fold 6
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[9]	training's pr_auc: 0.65534	valid_1's pr_auc: 0.2
average precision score 

In [191]:
feature_importance_df.sort_values("Cv", ascending=True).reset_index(drop=True)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Fold_6,Average,Std,Cv
0,intensity_std,65,23,45,28,50,6,36.166667,19.368502,0.535535
1,intensity_mean,21,7,19,10,21,1,13.166667,7.668478,0.582416
2,params2,113,40,47,25,58,12,49.166667,32.163212,0.654167
3,params3,166,63,83,32,95,13,75.333333,49.297284,0.654389
4,intensity_max,173,38,75,35,113,12,74.333333,54.746892,0.736505
5,params1,150,42,50,29,95,7,62.166667,47.418058,0.762757
6,layout_y,33,11,18,3,33,0,16.333333,13.110640,0.802692
7,intensity_min,75,13,25,19,38,0,28.333333,23.830419,0.841074
8,beta,79,13,42,8,58,1,33.500000,28.488302,0.850397
9,rms,68,16,32,4,46,1,27.833333,23.765638,0.853855


# submission

In [192]:
final_pred = pred_value
score = metric
sample_submission["target"] = final_pred
sample_submission.to_csv("../../../20200529atma/result/atmacup3_sample_submission"+str(score)[:-10]+".csv", index = False)

In [193]:
sample_submission.head()

,target
0,0.006489
1,0.006439
2,0.007080
3,0.007094
4,0.017002
